In [113]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import sklearn.model_selection
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

In [131]:
dataset_path = 'Admission_Predict_Ver1.1.csv'
raw_dataset = pd.read_csv(dataset_path, sep=",")

dataset = raw_dataset.copy()
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
Serial No.           500 non-null int64
GRE Score            500 non-null int64
TOEFL Score          500 non-null int64
University Rating    500 non-null int64
SOP                  500 non-null float64
LOR                  500 non-null float64
CGPA                 500 non-null float64
Research             500 non-null int64
Chance of Admit      500 non-null float64
dtypes: float64(4), int64(5)
memory usage: 35.2 KB


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [163]:
dataset = raw_dataset.copy() # Borrar el raw copy, es para poder hacer prints sobre esto y no tener datos distintos

# Split into vectors the dataset. y_all being the chance, x_all the other cols
# Y also gets categorized in 5 classes as output asks for.
y_all = keras.utils.to_categorical(dataset.pop("Chance of Admit "), num_classes=5)
x_all = dataset

# Split dataset into train and tests. 33% used as asked.
x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x_all, y_all, test_size=0.33, random_state=0)

In [171]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=tuple([x_train.shape[1]])),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(5, activation=tf.nn.sigmoid)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [172]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 64)                576       
_________________________________________________________________
dense_79 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_80 (Dense)             (None, 5)                 325       
Total params: 5,061
Trainable params: 5,061
Non-trainable params: 0
_________________________________________________________________


In [173]:
model.fit(x_train, y_train, batch_size=1024, epochs=200, validation_data=(x_valid, y_valid), verbose=1)

Train on 335 samples, validate on 165 samples
Epoch 1/200
335/335 [==============================] - 0s 1ms/sample - loss: 0.2238 - mean_absolute_error: 0.2402 - mean_squared_error: 0.2238 - val_loss: 0.0647 - val_mean_absolute_error: 0.0696 - val_mean_squared_error: 0.0647
Epoch 2/200
335/335 [==============================] - 0s 20us/sample - loss: 0.0522 - mean_absolute_error: 0.0573 - mean_squared_error: 0.0522 - val_loss: 0.0061 - val_mean_absolute_error: 0.0098 - val_mean_squared_error: 0.0061
Epoch 3/200
335/335 [==============================] - 0s 18us/sample - loss: 0.0030 - mean_absolute_error: 0.0050 - mean_squared_error: 0.0030 - val_loss: 5.6697e-07 - val_mean_absolute_error: 8.4377e-05 - val_mean_squared_error: 5.6697e-07
Epoch 4/200
335/335 [==============================] - 0s 31us/sample - loss: 2.1010e-07 - mean_absolute_error: 3.6330e-05 - mean_squared_error: 2.1010e-07 - val_loss: 5.6423e-07 - val_mean_absolute_error: 8.4173e-05 - val_mean_squared_error: 5.6423e-07

Epoch 34/200
335/335 [==============================] - 0s 43us/sample - loss: 1.5518e-07 - mean_absolute_error: 3.1297e-05 - mean_squared_error: 1.5518e-07 - val_loss: 4.1351e-07 - val_mean_absolute_error: 7.2095e-05 - val_mean_squared_error: 4.1351e-07
Epoch 35/200
335/335 [==============================] - 0s 31us/sample - loss: 1.5245e-07 - mean_absolute_error: 3.1026e-05 - mean_squared_error: 1.5245e-07 - val_loss: 4.0611e-07 - val_mean_absolute_error: 7.1449e-05 - val_mean_squared_error: 4.0611e-07
Epoch 36/200
335/335 [==============================] - 0s 26us/sample - loss: 1.4968e-07 - mean_absolute_error: 3.0747e-05 - mean_squared_error: 1.4968e-07 - val_loss: 3.9859e-07 - val_mean_absolute_error: 7.0786e-05 - val_mean_squared_error: 3.9859e-07
Epoch 37/200
335/335 [==============================] - 0s 28us/sample - loss: 1.4687e-07 - mean_absolute_error: 3.0461e-05 - mean_squared_error: 1.4687e-07 - val_loss: 3.9096e-07 - val_mean_absolute_error: 7.0108e-05 - val_mean_square

Epoch 67/200
335/335 [==============================] - 0s 31us/sample - loss: 6.0522e-08 - mean_absolute_error: 1.9711e-05 - mean_squared_error: 6.0522e-08 - val_loss: 1.6024e-07 - val_mean_absolute_error: 4.4977e-05 - val_mean_squared_error: 1.6024e-07
Epoch 68/200
335/335 [==============================] - 0s 22us/sample - loss: 5.8180e-08 - mean_absolute_error: 1.9334e-05 - mean_squared_error: 5.8180e-08 - val_loss: 1.5406e-07 - val_mean_absolute_error: 4.4107e-05 - val_mean_squared_error: 1.5406e-07
Epoch 69/200
335/335 [==============================] - 0s 39us/sample - loss: 5.5899e-08 - mean_absolute_error: 1.8958e-05 - mean_squared_error: 5.5899e-08 - val_loss: 1.4803e-07 - val_mean_absolute_error: 4.3241e-05 - val_mean_squared_error: 1.4803e-07
Epoch 70/200
335/335 [==============================] - 0s 33us/sample - loss: 5.3680e-08 - mean_absolute_error: 1.8586e-05 - mean_squared_error: 5.3680e-08 - val_loss: 1.4218e-07 - val_mean_absolute_error: 4.2382e-05 - val_mean_square

Epoch 100/200
335/335 [==============================] - 0s 37us/sample - loss: 1.3433e-08 - mean_absolute_error: 9.4574e-06 - mean_squared_error: 1.3433e-08 - val_loss: 3.6049e-08 - val_mean_absolute_error: 2.1477e-05 - val_mean_squared_error: 3.6049e-08
Epoch 101/200
335/335 [==============================] - 0s 25us/sample - loss: 1.2780e-08 - mean_absolute_error: 9.2309e-06 - mean_squared_error: 1.2780e-08 - val_loss: 3.4318e-08 - val_mean_absolute_error: 2.0962e-05 - val_mean_squared_error: 3.4318e-08
Epoch 102/200
335/335 [==============================] - 0s 29us/sample - loss: 1.2157e-08 - mean_absolute_error: 9.0110e-06 - mean_squared_error: 1.2157e-08 - val_loss: 3.2666e-08 - val_mean_absolute_error: 2.0458e-05 - val_mean_squared_error: 3.2666e-08
Epoch 103/200
335/335 [==============================] - 0s 125us/sample - loss: 1.1562e-08 - mean_absolute_error: 8.7942e-06 - mean_squared_error: 1.1562e-08 - val_loss: 3.1088e-08 - val_mean_absolute_error: 1.9964e-05 - val_mean_s

Epoch 132/200
335/335 [==============================] - 0s 21us/sample - loss: 2.6155e-09 - mean_absolute_error: 4.3070e-06 - mean_squared_error: 2.6155e-09 - val_loss: 7.1790e-09 - val_mean_absolute_error: 9.7415e-06 - val_mean_squared_error: 7.1790e-09
Epoch 133/200
335/335 [==============================] - 0s 21us/sample - loss: 2.4836e-09 - mean_absolute_error: 4.2032e-06 - mean_squared_error: 2.4836e-09 - val_loss: 6.8217e-09 - val_mean_absolute_error: 9.5033e-06 - val_mean_squared_error: 6.8217e-09
Epoch 134/200
335/335 [==============================] - 0s 24us/sample - loss: 2.3581e-09 - mean_absolute_error: 4.1006e-06 - mean_squared_error: 2.3581e-09 - val_loss: 6.4816e-09 - val_mean_absolute_error: 9.2718e-06 - val_mean_squared_error: 6.4816e-09
Epoch 135/200
335/335 [==============================] - 0s 22us/sample - loss: 2.2390e-09 - mean_absolute_error: 4.0012e-06 - mean_squared_error: 2.2390e-09 - val_loss: 6.1590e-09 - val_mean_absolute_error: 9.0438e-06 - val_mean_sq

335/335 [==============================] - 0s 21us/sample - loss: 5.0330e-10 - mean_absolute_error: 1.9976e-06 - mean_squared_error: 5.0330e-10 - val_loss: 1.4094e-09 - val_mean_absolute_error: 4.4773e-06 - val_mean_squared_error: 1.4094e-09
Epoch 165/200
335/335 [==============================] - 0s 21us/sample - loss: 4.7844e-10 - mean_absolute_error: 1.9519e-06 - mean_squared_error: 4.7844e-10 - val_loss: 1.3405e-09 - val_mean_absolute_error: 4.3730e-06 - val_mean_squared_error: 1.3405e-09
Epoch 166/200
335/335 [==============================] - 0s 19us/sample - loss: 4.5482e-10 - mean_absolute_error: 1.9073e-06 - mean_squared_error: 4.5482e-10 - val_loss: 1.2747e-09 - val_mean_absolute_error: 4.2716e-06 - val_mean_squared_error: 1.2747e-09
Epoch 167/200
335/335 [==============================] - 0s 30us/sample - loss: 4.3240e-10 - mean_absolute_error: 1.8644e-06 - mean_squared_error: 4.3240e-10 - val_loss: 1.2125e-09 - val_mean_absolute_error: 4.1735e-06 - val_mean_squared_error: 1

Epoch 197/200
335/335 [==============================] - 0s 41us/sample - loss: 1.0158e-10 - mean_absolute_error: 9.7867e-07 - mean_squared_error: 1.0158e-10 - val_loss: 2.8436e-10 - val_mean_absolute_error: 2.1572e-06 - val_mean_squared_error: 2.8436e-10
Epoch 198/200
335/335 [==============================] - 0s 41us/sample - loss: 9.7126e-11 - mean_absolute_error: 9.5958e-07 - mean_squared_error: 9.7126e-11 - val_loss: 2.7158e-10 - val_mean_absolute_error: 2.1133e-06 - val_mean_squared_error: 2.7158e-10
Epoch 199/200
335/335 [==============================] - 0s 24us/sample - loss: 9.2873e-11 - mean_absolute_error: 9.4077e-07 - mean_squared_error: 9.2873e-11 - val_loss: 2.5954e-10 - val_mean_absolute_error: 2.0704e-06 - val_mean_squared_error: 2.5954e-10
Epoch 200/200
335/335 [==============================] - 0s 37us/sample - loss: 8.8825e-11 - mean_absolute_error: 9.2295e-07 - mean_squared_error: 8.8825e-11 - val_loss: 2.4799e-10 - val_mean_absolute_error: 2.0298e-06 - val_mean_sq

In [174]:
x_test = dataset
y_test = model.predict(x_test)

y_test

array([[1.0000000e+00, 1.7881393e-07, 9.8317862e-05, 0.0000000e+00,
        7.0244074e-05],
       [1.0000000e+00, 1.2218952e-06, 1.8116832e-04, 0.0000000e+00,
        2.8848648e-05],
       [1.0000000e+00, 1.8477440e-06, 2.4309754e-04, 0.0000000e+00,
        2.8640032e-05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [1.0000000e+00, 1.7598343e-23, 0.0000000e+00, 0.0000000e+00,
        3.7036933e-29]], dtype=float32)